## DEMO

- Load CSV
- Data Validation & preprocessing
- EDA + Visulaization
- Bias Detection (data bias + prediction bias)
- Mitigation: Reweighting, Oversampling, Threshold Optimization
- Before and After Comparison
- Save dataset + retrained model 

### 1. Imports & Configurations

In [1]:
RANDOM_STATE = 42

DATA_PATH = "healthcare_dataset.csv" 

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    demographic_parity_difference,
    equalized_odds_difference,
    equal_opportunity_difference,
    demographic_parity_ratio,
    # disparate_impact_ratio,
)

from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.postprocessing import ThresholdOptimizer
from imblearn.over_sampling import RandomOverSampler

### 2. Load Dataset

In [3]:
df_raw = pd.read_csv(DATA_PATH)
print("Raw Shape:", df_raw.shape)

df_raw.head()

Raw Shape: (55500, 15)


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


### 3. Data Cleaning and Processing

In [4]:
df = df_raw.copy()

In [5]:
# drop duplicates
df = df.drop_duplicates()
print("After dropping duplicates:", df.shape)

After dropping duplicates: (54966, 15)


In [6]:
# drop rows with missing target
target_col = "Test Results"
df = df[df[target_col].notna()]

valid_target_values = ["Normal", "Abnormal"]
df = df[df[target_col].isin(valid_target_values)].copy()

In [7]:
# map to binary
target_mapping = {"Normal": 0, "Abnormal": 1}
df[target_col] = df[target_col].map(target_mapping).astype(int)

print("After cleaning target:", df.shape)
print(df[target_col].value_counts(dropna=False))

After cleaning target: (36768, 15)
Test Results
1    18437
0    18331
Name: count, dtype: int64


In [8]:
# handling missing values
numeric_cols_all = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols_all = df.select_dtypes(include=["object","category"]).columns.tolist() 

numeric_cols_all = [c for c in numeric_cols_all if c != target_col]

for col in numeric_cols_all:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
    
for col in categorical_cols_all:
    mode_val = df[col].mode().iloc[0]
    df[col] = df[col].fillna(mode_val)
    
print("Any remaining missing values?", df.isna().any().any())

Any remaining missing values? False


### 4. Identify Sensitive Attribute

In [9]:
def dataset_bias_scan(df, target_col, max_categories=10, dpd_threshold=0.1, di_threshold=0.8):
    """
        Scan categorical columns as potential sensitive attributes and compute dataset-level fairness metrics:
            - Demographic Parity Difference (based on target rates)
            - Disparate Impact Ratio
        Returns:
            metrics_by_col: dict[col] -> dict of metrics
            recommended_col: column with largest dpd (most disparity)
    """
    
    metrics_by_col = {}
    candidate_cols = [
        c for c in df.select_dtypes(include=['object','category']).columns
        if c != target_col and df[c].nunique() >= 2 and df[c].nunique() <= max_categories
    ]
    
    print("Candidate sensitive attributes:", candidate_cols)
    
    for col in candidate_cols:
        grouped = df.groupby(col)[target_col].mean()
        pos_rates = grouped.values
        
        if len(pos_rates) < 2:
            continue
        
        max_rate = pos_rates.max()
        min_rate = pos_rates.min()
        
        dpd = float(max_rate - min_rate)
        di = float(min_rate / max_rate) if max_rate > 0 else np.nan
        
        metrics_by_col[col] = {
            "positive_rates": grouped.to_dict(),
            "demographic_parity_difference": dpd,
            "disparate_impact_ratio": di,
            "potential_bias": (dpd >= dpd_threshold) or (di < di_threshold),
        }
        
    # column with largest dpd as most "biased"
    recommended_col = None
    if metrics_by_col:
        recommended_col = max(
            metrics_by_col.keys(),
            key = lambda c: metrics_by_col[c]['demographic_parity_difference']
        )
        
    return metrics_by_col, recommended_col

In [10]:
metrics_by_col, recommended_sensitive =  dataset_bias_scan(df, target_col)

print("\nDataset-level bias scan:")
for col, m in metrics_by_col.items():
    print(f"\nAttribute: {col}")
    print("  Positive rates:", m["positive_rates"])
    print("  DPD:", round(m["demographic_parity_difference"], 4))
    print("  DI :", round(m["disparate_impact_ratio"], 4))
    print("  Potential bias:", m["potential_bias"])
    
print("\nRecommended sensitive attribute based on disparity:", recommended_sensitive)

Candidate sensitive attributes: ['Gender', 'Blood Type', 'Medical Condition', 'Insurance Provider', 'Admission Type', 'Medication']

Dataset-level bias scan:

Attribute: Gender
  Positive rates: {'Female': 0.5045561193866972, 'Male': 0.4983460766769698}
  DPD: 0.0062
  DI : 0.9877
  Potential bias: False

Attribute: Blood Type
  Positive rates: {'A+': 0.504359197907585, 'A-': 0.5005412426932236, 'AB+': 0.49879728843210147, 'AB-': 0.4967741935483871, 'B+': 0.5028659611992945, 'B-': 0.5024864864864865, 'O+': 0.5042271840450899, 'O-': 0.5015337423312883}
  DPD: 0.0076
  DI : 0.985
  Potential bias: False

Attribute: Medical Condition
  Positive rates: {'Arthritis': 0.5126705653021443, 'Asthma': 0.48884514435695536, 'Cancer': 0.5058130014737187, 'Diabetes': 0.5056524547803618, 'Hypertension': 0.489396679270097, 'Obesity': 0.5060319530485817}
  DPD: 0.0238
  DI : 0.9535
  Potential bias: False

Attribute: Insurance Provider
  Positive rates: {'Aetna': 0.5010426803837064, 'Blue Cross': 0.495

In [11]:
# Choose final sensitive attribute:
if "Gender" in df.columns:
    sensitive_col = "Gender"
else:
    sensitive_col = recommended_sensitive

print("\nUsing sensitive attribute:", sensitive_col)


Using sensitive attribute: Gender


### 5. Define Features

In [12]:
# columns to drop
columns_to_drop = [
    "Name",
    "Doctor",
    "Hospital",
    "Date of Admission",
    "Discharge Date",
]

columns_to_drop = [c for c in columns_to_drop if c in df.columns]

feature_cols = [c for c in df.columns if c not in columns_to_drop + [target_col, sensitive_col]]

print("Feature columns:", feature_cols)

Feature columns: ['Age', 'Blood Type', 'Medical Condition', 'Insurance Provider', 'Billing Amount', 'Room Number', 'Admission Type', 'Medication']


In [13]:
X = df[feature_cols].copy()
y = df[target_col].copy()
sensitive_series = df[sensitive_col].copy()

### 6. Train and Test Split

In [14]:
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, sensitive_series, test_size=0.3,random_state=RANDOM_STATE, stratify=y)

print("Train Size:", X_train.shape)
print("Test Size:", X_test.shape)

Train Size: (25737, 8)
Test Size: (11031, 8)


### 7. Baseline RandomForest Model

In [15]:
categorical_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ],
    remainder="passthrough"
)

Numeric features: ['Age', 'Billing Amount', 'Room Number']
Categorical features: ['Blood Type', 'Medical Condition', 'Insurance Provider', 'Admission Type', 'Medication']


In [16]:
baseline_estimator = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=RANDOM_STATE, n_jobs=-1, max_features="sqrt")

baseline_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", baseline_estimator)
    ]
)

baseline_model.fit(X_train, y_train)

y_pred_base = baseline_model.predict(X_test)
acc_base = accuracy_score(y_test, y_pred_base)

print("Baseline accuracy:", acc_base)

Baseline accuracy: 0.5572477563230894


### 8. Fairness Metrics Helper Function

In [17]:
def compute_fairness_metrics(y_true, y_pred, sensitive_features, label=""):
    metrics = {
        "selection_rate": selection_rate,
        "accuracy": accuracy_score
    }
    
    mf = MetricFrame(
        metrics=metrics,
        y_true = y_true,
        y_pred = y_pred,
        sensitive_features=sensitive_features
    )
    
    dpd = demographic_parity_difference(
        y_true=y_true,
        y_pred = y_pred,
        sensitive_features=sensitive_features
    )
    
    eod = equalized_odds_difference(
        y_true=y_true,
        y_pred = y_pred,
        sensitive_features=sensitive_features
    )
    
    eopp = equal_opportunity_difference(
        y_true=y_true,
        y_pred = y_pred,
        sensitive_features=sensitive_features
    )
    
    di = demographic_parity_ratio(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    
    print(f"\n=== Fairness metrics: {label} ===")
    print("By-group selection rate:\n", mf.by_group["selection_rate"])
    print("By-group accuracy:\n", mf.by_group["accuracy"])
    print("Overall accuracy:", mf.overall["accuracy"])
    print("Demographic Parity Difference:", round(dpd, 4))
    print("Equalized Odds Difference:", round(eod, 4))
    print("Equal Opportunity Difference:", round(eopp, 4))
    print("Disparate Impact Ratio:", round(di, 4))
    
    return {
        "metric_frame": mf,
        "dpd": dpd,
        "eod": eod,
        "eopp": eopp,
        "di": di,
        "accuracy": mf.overall["accuracy"],
    }

In [18]:
baseline_fairness = compute_fairness_metrics(y_test, y_pred_base, s_test, label="Baseline RandomForest")


=== Fairness metrics: Baseline RandomForest ===
By-group selection rate:
 Gender
Female    0.519234
Male      0.500182
Name: selection_rate, dtype: float64
By-group accuracy:
 Gender
Female    0.552826
Male      0.561704
Name: accuracy, dtype: float64
Overall accuracy: 0.5572477563230894
Demographic Parity Difference: 0.0191
Equalized Odds Difference: 0.0281
Equal Opportunity Difference: 0.0103
Disparate Impact Ratio: 0.9633


### 9. Mitigation 1: Reweighting

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
reweight_constraint = DemographicParity()

log_reg = LogisticRegression(
    max_iter=300,
    solver="saga",
    n_jobs=-1
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_train_transformed = X_train_transformed.toarray()

reweight_mitigator = ExponentiatedGradient(
    estimator=log_reg,
    constraints=reweight_constraint
)

reweight_mitigator.fit(
    X_train_transformed, 
    y_train, 
    sensitive_features=s_train
)

X_test_transformed = preprocessor.transform(X_test)
X_test_transformed = X_test_transformed.toarray()

y_pred_reweight = reweight_mitigator.predict(X_test_transformed)

acc_reweight = accuracy_score(y_test, y_pred_reweight)

reweight_fairness = compute_fairness_metrics(
    y_test, y_pred_reweight, s_test, label="Reweighting"
)

/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: T


=== Fairness metrics: Reweighting ===
By-group selection rate:
 Gender
Female    0.735777
Male      0.740444
Name: selection_rate, dtype: float64
By-group accuracy:
 Gender
Female    0.505689
Male      0.501274
Name: accuracy, dtype: float64
Overall accuracy: 0.5034901640830387
Demographic Parity Difference: 0.0047
Equalized Odds Difference: 0.0097
Equal Opportunity Difference: 0.0003
Disparate Impact Ratio: 0.9937


### 10. Mitigation 2: Oversampling

In [21]:
# combine X, y, s for resampling
train_df = X_train.copy()
train_df['target'] = y_train.values
train_df['sensitive'] = s_train.values 

In [22]:
ros = RandomOverSampler(random_state=RANDOM_STATE)

train_resampled, y_train_over = ros.fit_resample(
    train_df.drop(columns=['target']), train_df['target']
)

X_train_over = train_resampled.drop(columns=['sensitive'])
s_train_over = train_resampled['sensitive']

print("Oversampled train shape:", X_train_over.shape)
print("Oversampled target distribution:\n", y_train_over.value_counts())

Oversampled train shape: (25812, 8)
Oversampled target distribution:
 target
0    12906
1    12906
Name: count, dtype: int64


In [23]:
# new RandomForest pipeline on oversampled data
oversample_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        (
            "clf",
            RandomForestClassifier(
                n_estimators=200,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
        )
    ]
)

oversample_model.fit(X_train_over, y_train_over)
y_pred_over = oversample_model.predict(X_test)
acc_over = accuracy_score(y_test, y_pred_over)

oversample_fairness = compute_fairness_metrics(
    y_test, y_pred_over, s_test, label="Oversampling"
)


=== Fairness metrics: Oversampling ===
By-group selection rate:
 Gender
Female    0.506592
Male      0.486531
Name: selection_rate, dtype: float64
By-group accuracy:
 Gender
Female    0.556800
Male      0.558609
Name: accuracy, dtype: float64
Overall accuracy: 0.5577010243858218
Demographic Parity Difference: 0.0201
Equalized Odds Difference: 0.0221
Equal Opportunity Difference: 0.0183
Disparate Impact Ratio: 0.9604


### 11. Mitigation 3: Threshold Optimization

In [24]:
thresh_model = ThresholdOptimizer(
    estimator=baseline_model,
    constraints="demographic_parity",
    prefit=True
)

thresh_model.fit(X_train, y_train, sensitive_features=s_train)

y_pred_thresh = thresh_model.predict(X_test, sensitive_features=s_test)

acc_thresh = accuracy_score(y_test, y_pred_thresh)

thresh_fairness = compute_fairness_metrics(y_test, y_pred_thresh, s_test, label="Threshold Optimization")


=== Fairness metrics: Threshold Optimization ===
By-group selection rate:
 Gender
Female    0.361026
Male      0.551147
Name: selection_rate, dtype: float64
By-group accuracy:
 Gender
Female    0.564385
Male      0.559884
Name: accuracy, dtype: float64
Overall accuracy: 0.5621430514005983
Demographic Parity Difference: 0.1901
Equalized Odds Difference: 0.1949
Equal Opportunity Difference: 0.1851
Disparate Impact Ratio: 0.655


### 12. Compare Accuracy Accross All models

In [25]:
print("\n=== Accuracy Comparison ===")
print(f"Baseline RandomForest           : {acc_base:.4f}")
print(f"Reweighting (ExponentiatedGrad): {acc_reweight:.4f}")
print(f"Oversampling (RandomOverSampler): {acc_over:.4f}")
print(f"Threshold Optimization          : {acc_thresh:.4f}")


=== Accuracy Comparison ===
Baseline RandomForest           : 0.5572
Reweighting (ExponentiatedGrad): 0.5035
Oversampling (RandomOverSampler): 0.5577
Threshold Optimization          : 0.5621


### 13. Choose a Final Model:

- smallest[dpd]
- with accuracy not too far from baseline

In [26]:
def pick_best_model(models_dict):
    # compute baseline acc
    baseline_acc = models_dict['baseline']['accuracy']
    
    best_name = "baseline"
    best_score = None
    
    for name, stats in models_dict.items():
        dpd = abs(stats['dpd'])
        acc = stats['accuracy']
        
        # penalize model that loss 5% acc
        acc_penalty = max(0.0, baseline_acc - acc - 0.05)
        
        composite = dpd  + acc_penalty # lower the better
        
        if best_score is None or composite < best_score:
            best_name = name
            best_score = composite
            
    return best_name

In [27]:
models_stats = {
    "baseline": baseline_fairness,
    "reweight": reweight_fairness,
    "oversample": oversample_fairness,
    "threshold": thresh_fairness,
}

best_model_name = pick_best_model(models_stats)
print("\nChosen best model (fairness vs accuracy trade-off):", best_model_name)


Chosen best model (fairness vs accuracy trade-off): reweight


In [28]:
model_objects = {
    "baseline": baseline_model,
    "reweight": reweight_mitigator,
    "oversample": oversample_model,
    "threshold": thresh_model,
}

final_debiased_model = model_objects[best_model_name]

### 14. Save debiased dataset & model

In [29]:
OUTPUT_DIR = Path("health_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# debiased dataset
debiased_dataset = X_train_over.copy()
debiased_dataset[target_col] = y_train_over.values
debiased_dataset[sensitive_col] = s_train_over.values

debiased_dataset_path = OUTPUT_DIR / "debiased_dataset.csv"
debiased_dataset.to_csv(debiased_dataset_path, index=False)
print("Saved debiased dataset to:", debiased_dataset_path)

# Save models
baseline_model_path = OUTPUT_DIR / "baseline_rf_model.joblib"
debiased_model_path = OUTPUT_DIR / "debiased_rf_model.joblib"

joblib.dump(baseline_model, baseline_model_path)
joblib.dump(final_debiased_model, debiased_model_path)

print("Saved baseline model to:", baseline_model_path)
print("Saved debiased model to:", debiased_model_path)

Saved debiased dataset to: health_outputs/debiased_dataset.csv
Saved baseline model to: health_outputs/baseline_rf_model.joblib
Saved debiased model to: health_outputs/debiased_rf_model.joblib


### Test

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [31]:
logistic_model = LogisticRegression(
    max_iter=500,
    solver="saga",            # supports sparse data
    n_jobs=-1
)

rf_model = RandomForestClassifier(
    n_estimators=150,
    max_depth=20,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

xgb_model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

In [34]:
# If already transformed data exists
X_train_dense = preprocessor.fit_transform(X_train)
X_train_dense = X_train_dense.toarray()

X_test_dense = preprocessor.transform(X_test)
X_test_dense = X_test_dense.toarray()

logistic_model.fit(X_train_dense, y_train)
rf_model.fit(X_train_dense, y_train)          # RF requires numeric input
xgb_model.fit(X_train_dense, y_train)

/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [36]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

models = {
    "Logistic": logistic_model,
    "RandomForest": rf_model,
    "XGBoost": xgb_model,
}

results = []

for name, model in models.items():
    y_pred = model.predict(X_test_dense)   # use processed numeric features
    acc = accuracy_score(y_test, y_pred)
    dpd = demographic_parity_difference(y_test, y_pred, sensitive_features=s_test)
    eod = equalized_odds_difference(y_test, y_pred, sensitive_features=s_test)
    results.append([name, acc, dpd, eod])

import pandas as pd
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "DPD", "EOD"])
print(df_results)

          Model  Accuracy       DPD       EOD
0      Logistic  0.503671  0.005332  0.006316
1  RandomForest  0.552081  0.018869  0.029521
2       XGBoost  0.522709  0.002837  0.016757
